# Voldemort's Stable Diffusion WebUI

Adapted from: https://colab.research.google.com/drive/1Iy-xW9t1-OQWhb0hNxueGij8phCyluOh

**Updating the Notebook**

In the Tools section at the bottom of the notebook there is a block to download the latest version from Github.

**Guides**
- [Getting started on Paperspace](https://github.com/Engineer-of-Stuff/stable-diffusion-paperspace/blob/main/Docs/Paperspace%20Guide%20for%20Retards.md)
- [Using the WebUI](https://rentry.org/voldy)
- [Using the Inpainter](https://rentry.org/drfar)
- [Textual Inversion](https://rentry.org/aikgx)
- [Crowd-Sourced Prompts](https://lexica.art/)
- [Artist Name Prompts](https://sgreens.notion.site/sgreens/4ca6f4e229e24da6845b6d49e6b08ae7?v=fdf861d1c65d456e98904fe3f3670bd3)
- [Stable Diffusion Models](https://cyberes.github.io/stable-diffusion-models)
- [Textual Inversion Models](https://cyberes.github.io/stable-diffusion-textual-inversion-models/)
- [Have I Been Trained?](https://haveibeentrained.com/)

## Installation and Setup

You must reinstall everything each time you restart the machine. If already downloaded, dependencies will be auto-updated.

**Where to store the models**

`/storage/` is persistent storage shared across all machines on your account.

`/notebooks/` is storage for this notebook only.

We're going to store models in `/storage/models` and create a symlink.

_You must run the block below or else the variable won't be accessable in the notebook._

In [ ]:
model_storage_dir = '/storage/models' # no trailing slash

# ===============================================================
# Save the variable to Jupiter's temp storage to access it when the kernel restarts.
# To reset your storage directory, rerun this cell.
%store model_storage_dir

### Clone the central repository

In [ ]:
import os.path
%store -r model_storage_dir
if not os.path.exists('/notebooks/stable-diffusion-webui'):
    %cd /notebooks/
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
else: # update repo if already exists
    print('stable-diffusion-webui already downloaded, updating...')
    %cd /notebooks/stable-diffusion-webui
    !git pull

### Install requirements and download repositories

In [ ]:
%cd /notebooks/stable-diffusion-webui
%store -r model_storage_dir

# Import launch.py which will automatically run the install script but not launch the WebUI.
# They require a few specific external git repo commits so we have to do it their way. 
import launch

# Download the GFPGAN face restorer.
if not os.path.exists('/notebooks/stable-diffusion-webui/GFPGANv1.3.pth'):
    !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -O /notebooks/stable-diffusion-webui/GFPGANv1.3.pth
else:
    print('GFPGANv1.3.pth already downloaded')
    
# Make sure your models storage directory exists
!mkdir -p $model_storage_dir

# Link the output folder to /notebooks/outputs
!mkdir -p /notebooks/stable-diffusion-webui/outputs
!ln -s /notebooks/stable-diffusion-webui/outputs /notebooks/outputs

### Download the Model

I've provided a few different ways of aquiring the models. Try the torrent option first. You don't need to repeat this step if you've already downloaded the models.

There are a few additional models available here: https://cyberes.github.io/stable-diffusion-models

If you're interested in textual inversion, here's the database: https://cyberes.github.io/stable-diffusion-textual-inversion-models

**Filesize and Storage Disclaimer**

Paperspace free tier has only 5GB of storage space. There's really not a way to get around this limitation, but here's a few suggestions.
1. Add a payment method to your account. Storage overages are billed at \$0.29/GB and billing occurs monthly and runs at midnight on the first of each month. With a payment method on file, Paperspace will let you use more storage and if you time it right you shouldn't actually be charged for it.
2. Upgrade to a Pro account. They'll give you 15GB and you'll get longer runtimes and more powerful free GPUs.
3. Use my referral code `KQLRH37` You'll get \$10 credit that you should be able to put towards the storage overage charges. Redeem the code at the bottom of the Billing page.

If you're on free tier, only download one model.

**Torrent Instructions**

Aria2 may show some errors/warnings while downloading. Those are fine, when it eventually says "Download Complete" that means everything worked as it should.

#### Standard Model

**Torrent**

In [ ]:
%store -r model_storage_dir
!apt update
!apt install -y aria2
%cd $model_storage_dir
!aria2c --seed-time=0 --max-overall-upload-limit=1K "magnet:?xt=urn:btih:3A4A612D75ED088EA542ACAC52F9F45987488D1C&tr=udp://tracker.opentrackr.org:1337/announce"

**Web Download**

Voldy provided an alternative download if you don't want to use HuggingFace.

[https://drive.google.com/file/d/1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA/view](https://drive.google.com/file/d/1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA/view)

Download it to your computer then upload it to your model storage directory (make sure it's named `sd-v1-4.ckpt`).

HuggingFace is much faster and reliable but you need to get access to the repo and provide your user token.

In [ ]:
user_token = "<enter your user token here>"

# ===============================================================================================
%store -r model_storage_dir
!wget --header="'Authorization: Bearer {user_token}'" https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O $model_storage_dir/sd-v1-4.ckpt

#### Waifu Diffusion

The original Stable Diffusion anime finetune.

In [ ]:
!apt update
!apt install -y aria2
%cd $model_storage_dir
!aria2c --seed-time=0 --max-overall-upload-limit=1K "magnet:?xt=urn:btih:153590FD7E93EE11D8DB951451056C362E3A9150&dn=wd-v1-2-full-ema-pruned.ckpt&tr=udp://tracker.opentrackr.org:1337/announce"

#### trinart_stable_diffusion_v2

Anime finetune.

The 60000 steps version is the original, the 115000 steps is the 60000 with additional training. Use the 60000 step version if the style nudging is too much.

[See the comparison here.](https://cyberes.github.io/stable-diffusion-models/#model-comparison)

**60000**

In [ ]:
%store -r model_storage_dir
!wget https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step60000.ckpt -O $model_storage_dir/trinart2_step60000.ckpt

**95000**

In [ ]:
%store -r model_storage_dir
!wget https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step95000.ckpt -O $model_storage_dir/trinart2_step95000.ckpt

**115000**

In [ ]:
%store -r model_storage_dir
!wget https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step115000.ckpt -O $model_storage_dir/trinart2_step115000.ckpt

### Clean up and restart the kernel

In [ ]:
%store -r model_storage_dir

# Get some storage back
!pip cache purge
!rm $model_storage_dir/*.aria2
!apt remove --purge -y aria2 p7zip-full
!apt autoremove --purge -y
!apt clean

# Restart the kernel
import os
os.kill(os.getpid(), 9)

### Link the models directory

Create symlinks. The file will be stored in the models storage directory and linked to where the WebUI expects the files to be.

In [ ]:
%store -r model_storage_dir
import os

# Check for broken symlinks and remove them
for file in os.listdir('/notebooks/stable-diffusion-webui/models/'):
    path = f'/notebooks/stable-diffusion-webui/models/{file}'
    if os.path.islink(path) and not os.path.exists(os.readlink(path)):
        print(f'Symlink broken, removing: {file}')
        os.unlink(path)

# Make symlinks for new files
for file in os.listdir(model_storage_dir):
    if file.endswith("ckpt"):
        path = f'/notebooks/stable-diffusion-webui/models/{file}'
        if not os.path.exists(path):
            print(f'New model: {file}')
            !ln -s $model_storage_dir/$file /notebooks/stable-diffusion-webui/models/$file
        !ls -la --block-size=GB /notebooks/stable-diffusion-webui/models/$file

# Launch the WebUI

Run this block to launch the WebUI. You will get a link to nnn.gradio.app, that's your WebUI. Follow it.

If you have a lot of VRAM and aren't generating large images you can add the flag `--disable-opt-split-attention` to disable VRAM optimizations for a speed boost.

**Troubleshooting**

- If you have any issues, try restarting the kernel.
- `EOFError: Ran out of input` probably means you ran out of storage space and the model `.ckpt` file wasn't downloaded completely. Try cleaning up your files or you can reset your storage with the _Reset Storage_ block in the Tools section below.
- If you're having issues with your results not loading, try running an ngrok proxy. More details in the Tools section below.

In [ ]:
%store -r model_storage_dir
%cd /notebooks/stable-diffusion-webui
!python /notebooks/stable-diffusion-webui/webui.py --share # --disable-opt-split-attention

# Export Generations

This block will rename and compress the outputs with 7zip max compression. It expects you to have `log/` and `outputs/` in `/notebooks/stable-diffusion-webui/`.

In [ ]:
!apt update
!apt install -y p7zip-full
from datetime import datetime
datetime_str = datetime.now().strftime('%m-%d-%Y_%H:%M:%S')
%cd /notebooks/
!mkdir $datetime_str
!mv /notebooks/stable-diffusion-webui/log $datetime_str
!mv /notebooks/stable-diffusion-webui/outputs/* $datetime_str
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on {datetime_str}.7z {datetime_str}/

### Delete old output folder

This block will delete the folder you just compressed.

In [ ]:
!rm -rf /notebooks/$datetime_str/
!echo Deleted /notebooks/$datetime_str/

# Tools

### Show graphics card info

In [ ]:
!nvidia-smi

### Download the latest version of this notebook from Github

Run this and refresh the page (press F5). Don't save anything or you will overwrite the downloaded file.

In [ ]:
!mv /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb.backup # save your old notebook to a backup
!wget https://raw.githubusercontent.com/Engineer-of-Stuff/stable-diffusion-paperspace/main/StableDiffusionUI_Voldemort_paperspace.ipynb -O /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb

### Reset Repository

Sometimes AUTOMATIC1111 breaks something. Go to https://github.com/AUTOMATIC1111/stable-diffusion-webui/commits/master and choose a commit to revert to.

In [ ]:
!git reset --hard <commit>

### Delete all files and reset storage

This will delete ALL your files in `/notebooks/` and `/storage/`. Use if you're having issues with zero storage space and you don't want to delete your notebook.

In [ ]:
# Uncomment the lines below to run this block. You can highlight the lines and do ctrl + /
# !rm -rf /storage/*
# !mv /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb / # move the notebook out of the directory before we nuke it
# !rm -rf /notebooks/*
# !mv /StableDiffusionUI_Voldemort_paperspace.ipynb /notebooks/ # move it back

### ngrok Proxy

There's an issue with Gradio's infrastructure where you can't load an image larger than 2MB. We can get around this by running our own proxy with ngrok. Here's how to do it:

1. Add `--port 7860` to the `!python ... webui.py` line in the Launch WebUI section above.
2. Launch the WebUI.
3. Go to [ngrok.com](https://ngrok.com/) in a new tab and sign up. Don't close that tab yet.
4. Open the terminal by clicking on the terminal button in the left sidebar menu. If you're not sure which one that is, hover over the items and a little tooltip will appear.
5. Run this in the terminal. Make sure to replace `<your authtoken>` with your auth token.

```bash
pip install pyngrok
ngrok authtoken <your authtoken>
ngrok http 7860
```

ngrok will assign you a subdomain and display a link to it. That's the link to your WebUI instance. If you want, create a new file named `start-ngrok.sh` and copy those lines into it so you can quickly connect.